## Hypothesis: Flood experience has a positive effect on preparedness

In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df = pd.read_csv(path)
df.head()

,id,state,zipcode,geographic_division,census_region,county,experience,supplies,insured,involved,...,sex,education,race,homeownership,income,rentmortgage,rurality,hazard_weight,geometry,zip_count
0,1,Washington,98033,Pacific,West,King,0.0,0,0,1,...,0.0,4,White,1,87500,1250.0,1,0.255097,POLYGON ((-13608075.601186747 6055357.24645823...,1
1,2,Florida,33950,South Atlantic,South,Charlotte,1.0,0,1,1,...,1.0,5,White,1,175000,3000.0,1,2.836145,POLYGON ((-9139287.781401768 3112400.961508024...,1
2,3,New Jersey,7031,Middle Atlantic,Northeast,Bergen,1.0,0,0,1,...,1.0,1,White,0,47500,1250.0,1,1.165971,POLYGON ((-8254065.171859486 4980899.960122439...,1
3,5,Pennsylvania,19148,Middle Atlantic,Northeast,Philadelphia,1.0,0,1,0,...,0.0,4,White,1,125000,1250.0,1,0.624890,POLYGON ((-8368433.592186097 4851218.465580212...,1
4,7,Florida,32976,South Atlantic,South,Brevard,0.0,1,1,0,...,0.0,2,White,1,20000,250.0,1,0.695075,POLYGON ((-8971557.027329512 3234931.888104651...,1


#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication


In [3]:
# define the regression model 
def model (function, data):
    model = smf.logit(function, data=data).fit()
    print(model.summary()) # print model summary
    print('BIC:', model.bic) # print BIC additionally to defaults model fit parameters
    # print odds ratios for output = 0 and output = 1
    print('likelihood of adaptation when there is no flood experience = ', 1 / (1 + np.exp(-model.params['Intercept'])))
    print('likelihood of adaptation when there is flood experience = ', 1 / (1 + np.exp(-(model.params['Intercept']+model.params['experience']))))

    # compute odds ratio
    params = model.params
    conf = model.conf_int()
    odds_ratios = pd.DataFrame({
        "OR": np.exp(params),
        "Lower CI": np.exp(conf[0]),
        "Upper CI": np.exp(conf[1])
        })
    odds_ratios.index.name = 'Variable'
    odds_ratios.reset_index(inplace=True)
    # compute marginal effects
    marginal_effects = model.get_margeff().summary()

    return odds_ratios, marginal_effects 
    

In [4]:
def probit (function, data):
    model = smf.probit(function, data=data).fit()
    print(model.summary())
    print('BIC=', model.bic)
    marginal = model.get_margeff().summary()
    return marginal

In [5]:
odds, marg_eff = model('made_safer ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.466982
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02281
Time:                        18:05:54   Log-Likelihood:                -220.42
converged:                       True   LL-Null:                       -225.56
Covariance Type:            nonrobust   LLR p-value:                  0.001337
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7483      0.152    -11.523      0.000      -2.046      -1.451
experience     0.8101      0.

In [6]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.174061,0.129286,0.234343
1,experience,2.248082,1.381760,3.657561


In [7]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:             made_safer
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1190      0.036      3.333      0.001       0.049       0.189
==============================================================================
"""

In [8]:
odds, marg_eff = model('insured ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.499629
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                insured   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.03767
Time:                        18:05:54   Log-Likelihood:                -235.82
converged:                       True   LL-Null:                       -245.06
Covariance Type:            nonrobust   LLR p-value:                 1.733e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6376      0.146    -11.213      0.000      -1.924      -1.351
experience     1.0254      0.

In [9]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.194444,0.146043,0.258887
1,experience,2.788296,1.756422,4.426382


In [10]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                insured
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1652      0.036      4.597      0.000       0.095       0.236
==============================================================================
"""

In [11]:
odds, marg_eff = model('learned_routes ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.500705
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         learned_routes   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.06454
Time:                        18:05:54   Log-Likelihood:                -236.33
converged:                       True   LL-Null:                       -252.64
Covariance Type:            nonrobust   LLR p-value:                 1.125e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6809      0.148    -11.341      0.000      -1.971      -1.390
experience     1.3337      0.

In [12]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.186207,0.139264,0.248974
1,experience,3.795062,2.404971,5.988634


In [13]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:         learned_routes
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.2164      0.034      6.429      0.000       0.150       0.282
==============================================================================
"""

In [14]:
odds, marg_eff = model('supplies ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.480375
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               supplies   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08905
Time:                        18:05:54   Log-Likelihood:                -226.74
converged:                       True   LL-Null:                       -248.90
Covariance Type:            nonrobust   LLR p-value:                 2.771e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8192      0.156    -11.691      0.000      -2.124      -1.514
experience     1.5678      0.

In [15]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.162162,0.119537,0.219987
1,experience,4.796296,3.016831,7.625372


In [16]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               supplies
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.2418      0.031      7.793      0.000       0.181       0.303
==============================================================================
"""

In [17]:
odds, marg_eff = model('involved ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.273968
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               involved   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08732
Time:                        18:05:54   Log-Likelihood:                -129.31
converged:                       True   LL-Null:                       -141.68
Covariance Type:            nonrobust   LLR p-value:                 6.552e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0204      0.256    -11.797      0.000      -3.522      -2.519
experience     1.6535      0.

In [18]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.04878,0.029534,0.080570
1,experience,5.22549,2.697507,10.122587


In [19]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               involved
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1260      0.027      4.641      0.000       0.073       0.179
==============================================================================
"""

In [20]:
odds, marg_eff = model('made_plan ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.504336
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              made_plan   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.03871
Time:                        18:05:54   Log-Likelihood:                -238.05
converged:                       True   LL-Null:                       -247.63
Covariance Type:            nonrobust   LLR p-value:                 1.194e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6164      0.145    -11.147      0.000      -1.901      -1.332
experience     1.0384      0.

In [21]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.198606,0.149472,0.263892
1,experience,2.824561,1.784000,4.472057


In [22]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:              made_plan
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1694      0.036      4.700      0.000       0.099       0.240
==============================================================================
"""

In [23]:
odds, marg_eff = model('practiced_drills ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.267162
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       practiced_drills   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.04588
Time:                        18:05:54   Log-Likelihood:                -126.10
converged:                       True   LL-Null:                       -132.16
Covariance Type:            nonrobust   LLR p-value:                 0.0004967
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.8965      0.242    -11.963      0.000      -3.371      -2.422
experience     1.2101      0.

In [24]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.055215,0.034353,0.088746
1,experience,3.353909,1.711196,6.573593


In [25]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:       practiced_drills
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0870      0.026      3.349      0.001       0.036       0.138
==============================================================================
"""

In [26]:
odds, marg_eff = model('alerts ~ experience', data = df)

Optimization terminated successfully.
         Current function value: 0.472609
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 alerts   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02379
Time:                        18:05:54   Log-Likelihood:                -223.07
converged:                       True   LL-Null:                       -228.51
Covariance Type:            nonrobust   LLR p-value:                 0.0009759
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7255      0.151    -11.464      0.000      -2.021      -1.431
experience     0.8256      0.

In [27]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.178082,0.132587,0.239189
1,experience,2.283178,1.408885,3.700021


In [28]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 alerts
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1232      0.036      3.434      0.001       0.053       0.194
==============================================================================
"""

In [29]:
odds, marg_eff = model('family_communication ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.319640
         Iterations 6
                            Logit Regression Results                            
Dep. Variable:     family_communication   No. Observations:                  472
Model:                            Logit   Df Residuals:                      470
Method:                             MLE   Df Model:                            1
Date:                  Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02783
Time:                          18:05:54   Log-Likelihood:                -150.87
converged:                         True   LL-Null:                       -155.19
Covariance Type:              nonrobust   LLR p-value:                  0.003292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.5040      0.204    -12.276      0.000      -2.904      -2.104
experience   

In [30]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.081761,0.054818,0.121947
1,experience,2.538462,1.380883,4.666426


In [31]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects        
=======================================
Dep. Variable:     family_communication
Method:                            dydx
At:                             overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0834      0.028      2.942      0.003       0.028       0.139
==============================================================================
"""

## Let's repeat with probit

In [32]:
probit('made_safer ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.466982
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02281
Time:                        18:06:38   Log-Likelihood:                -220.42
converged:                       True   LL-Null:                       -225.56
Covariance Type:            nonrobust   LLR p-value:                  0.001337
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0439      0.083    -12.605      0.000      -1.206      -0.882
experience     0.4648      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:             made_safer
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1209      0.037      3.299      0.001       0.049       0.193
==============================================================================
"""

In [33]:
probit('insured ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.499629
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                insured   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.03767
Time:                        18:06:51   Log-Likelihood:                -235.82
converged:                       True   LL-Null:                       -245.06
Covariance Type:            nonrobust   LLR p-value:                 1.733e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9831      0.081    -12.153      0.000      -1.142      -0.825
experience     0.6019      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                insured
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1685      0.037      4.527      0.000       0.096       0.241
==============================================================================
"""

In [34]:
probit('learned_routes ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.500705
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:         learned_routes   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.06454
Time:                        18:06:57   Log-Likelihood:                -236.33
converged:                       True   LL-Null:                       -252.64
Covariance Type:            nonrobust   LLR p-value:                 1.125e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0070      0.082    -12.336      0.000      -1.167      -0.847
experience     0.7899      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:         learned_routes
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.2218      0.035      6.276      0.000       0.153       0.291
==============================================================================
"""

In [35]:
probit('supplies ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.480375
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:               supplies   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08905
Time:                        18:07:00   Log-Likelihood:                -226.74
converged:                       True   LL-Null:                       -248.90
Covariance Type:            nonrobust   LLR p-value:                 2.771e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0824      0.084    -12.867      0.000      -1.247      -0.918
experience     0.9251      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:               supplies
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.2486      0.033      7.560      0.000       0.184       0.313
==============================================================================
"""

In [36]:
probit('involved ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.273968
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:               involved   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08732
Time:                        18:07:05   Log-Likelihood:                -129.31
converged:                       True   LL-Null:                       -141.68
Covariance Type:            nonrobust   LLR p-value:                 6.552e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6797      0.117    -14.399      0.000      -1.908      -1.451
experience     0.8492      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:               involved
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1253      0.026      4.830      0.000       0.074       0.176
==============================================================================
"""

In [37]:
probit('made_plan ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.504336
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:              made_plan   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.03871
Time:                        18:07:11   Log-Likelihood:                -238.05
converged:                       True   LL-Null:                       -247.63
Covariance Type:            nonrobust   LLR p-value:                 1.194e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9713      0.081    -12.060      0.000      -1.129      -0.813
experience     0.6112      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:              made_plan
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1728      0.037      4.625      0.000       0.100       0.246
==============================================================================
"""

In [38]:
probit('practiced_drills ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.267162
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:       practiced_drills   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.04588
Time:                        18:07:17   Log-Likelihood:                -126.10
converged:                       True   LL-Null:                       -132.16
Covariance Type:            nonrobust   LLR p-value:                 0.0004967
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6227      0.112    -14.453      0.000      -1.843      -1.403
experience     0.6127      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:       practiced_drills
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0876      0.026      3.419      0.001       0.037       0.138
==============================================================================
"""

In [39]:
probit('alerts ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.472609
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                 alerts   No. Observations:                  472
Model:                         Probit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02379
Time:                        18:07:20   Log-Likelihood:                -223.07
converged:                       True   LL-Null:                       -228.51
Covariance Type:            nonrobust   LLR p-value:                 0.0009759
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0315      0.082    -12.516      0.000      -1.193      -0.870
experience     0.4753      0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 alerts
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.1252      0.037      3.397      0.001       0.053       0.198
==============================================================================
"""

In [40]:
probit('family_communication ~ experience', data=df)

Optimization terminated successfully.
         Current function value: 0.319640
         Iterations 6
                           Probit Regression Results                            
Dep. Variable:     family_communication   No. Observations:                  472
Model:                           Probit   Df Residuals:                      470
Method:                             MLE   Df Model:                            1
Date:                  Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02783
Time:                          18:07:29   Log-Likelihood:                -150.87
converged:                         True   LL-Null:                       -155.19
Covariance Type:              nonrobust   LLR p-value:                  0.003292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4354      0.100    -14.342      0.000      -1.632      -1.239
experience   

<class 'statsmodels.iolib.summary.Summary'>
"""
        Probit Marginal Effects        
=======================================
Dep. Variable:     family_communication
Method:                            dydx
At:                             overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0845      0.029      2.952      0.003       0.028       0.141
==============================================================================
"""

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication